In [2]:
import os
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from timm.models.layers import DropPath
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

# 1) Reproducibility & Device Setup
def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2) File paths
train_csv = '/kaggle/input/smai-project/Phase_2_data/labels_train.csv'
val_csv   = '/kaggle/input/smai-project/Phase_2_data/labels_val.csv'
train_dir = '/kaggle/input/smai-project/Phase_2_data/images_train'
val_dir   = '/kaggle/input/smai-project/Phase_2_data/images_val'

# 3) Dataset & DataLoader
class GeoAngleDataset(Dataset):
    def __init__(self, csv_file, img_dir, transforms=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.img_dir, row['filename'])).convert('RGB')
        if self.transforms:
            img = self.transforms(img)
        theta = math.radians(float(row['angle']))
        target = torch.tensor([math.cos(theta), math.sin(theta)], dtype=torch.float32)
        return img, target, row['filename']

# Transforms (224×224)
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(0.7,1.0)),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ColorJitter(0.3,0.3,0.3,0.15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

batch_size = 2
accum_steps = 2  # effective BS=4

dataset_train = GeoAngleDataset(train_csv, train_dir, transforms=train_transforms)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=4)
dataset_val = GeoAngleDataset(val_csv, val_dir, transforms=val_transforms)
loader_val   = DataLoader(dataset_val,   batch_size=batch_size, shuffle=False, num_workers=4)

# 4) Model Definition: DINOv2 ViT-B/14 + head

def freeze_module(module):
    for p in module.parameters(): p.requires_grad=False

class AngleModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Load DINOv2 ViT-B/14 from torch.hub
        self.backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14', pretrained=True)
        embed_dim = self.backbone.embed_dim  # should be 768

        # Freeze patch embedding and first 6 transformer blocks
        freeze_module(self.backbone.patch_embed)
        for blk in self.backbone.blocks[:6]:
            freeze_module(blk)
        # Apply DropPath to last 6 blocks
        for blk in self.backbone.blocks[6:]:
            blk.drop_path = DropPath(0.1)

        # Regression head: 768 → 384 → 2
        self.head = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(embed_dim, embed_dim // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(embed_dim // 2, 2)
        )

    def forward(self, x):
        feats = self.backbone(x)
        raw = self.head(feats)
        norm = raw.norm(dim=1, keepdim=True).clamp(min=1e-6)
        return raw / norm

model = AngleModel().to(device)

# 5) Loss, Metric, Optimizer & Scheduler
criterion = nn.MSELoss()
optimizer = optim.AdamW([
    {'params': [p for p in model.backbone.parameters() if p.requires_grad], 'lr':5e-6},
    {'params': model.head.parameters(), 'lr':5e-4}
], weight_decay=1e-2)

epochs = 100
scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-7)
scaler = GradScaler()

def angular_mae(pred, true):
    dots = (pred * true).sum(dim=1).clamp(-1,1)
    return (torch.acos(dots) * 180.0 / math.pi).mean().item()

# 6) Training & Validation Functions
def train_one_epoch(ep):
    model.train(); optimizer.zero_grad(); total_loss=0.0; total_mae=0.0
    for i, (imgs, targs, _) in enumerate(tqdm(loader_train, desc=f"Train {ep+1}")):
        imgs, targs = imgs.to(device), targs.to(device)
        with autocast():
            preds = model(imgs)
            loss = criterion(preds, targs) / accum_steps
        scaler.scale(loss).backward()
        total_loss += loss.item() * accum_steps
        total_mae += angular_mae(preds, targs)
        if (i+1) % accum_steps == 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer); scaler.update(); optimizer.zero_grad()
    scheduler.step()
    print(f"Epoch {ep+1} Train Loss {total_loss/len(dataset_train):.4f} MAE {total_mae/len(loader_train):.4f}")


def validate(ep):
    model.eval()
    val_loss = 0.0
    val_mae = 0.0
    all_preds = []
    fnames = []
    with torch.no_grad():
        for imgs, targs, names in tqdm(loader_val, desc="Val"):
            imgs, targs = imgs.to(device), targs.to(device)
            preds = model(imgs)
            val_loss += criterion(preds, targs).item() * imgs.size(0)
            val_mae += angular_mae(preds, targs) * imgs.size(0)
            all_preds.append(preds.cpu())
            fnames.extend(names)
    val_loss /= len(dataset_val)
    val_mae /= len(dataset_val)
    print(f"Epoch {ep+1} Val Loss {val_loss:.4f} MAE {val_mae:.4f}")
    # Save predictions
    preds_cat = torch.cat(all_preds)
    angles = (torch.atan2(preds_cat[:,1], preds_cat[:,0]) * 180.0 / math.pi) % 360
    pd.DataFrame({'filename': fnames, 'pred_angle': angles.numpy()}).to_csv('val_preds.csv', index=False)
    return val_loss, val_mae

# 7) Run training
best_mae = float('inf')
for epoch in range(epochs):
    train_one_epoch(epoch)
    v_loss, v_mae = validate(epoch)
    if v_mae < best_mae:
        best_mae = v_mae
        torch.save(model.state_dict(), 'best_angle_model.pth')
        pd.read_csv('val_preds.csv').to_csv('best_val_preds.csv', index=False)
print(f"Best Val MAE: {best_mae:.4f}")


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb

Epoch 1 Train Loss 0.4249 MAE 78.9646


Val: 100%|██████████| 185/185 [00:08<00:00, 21.88it/s]


Epoch 1 Val Loss 0.5665 MAE 57.7966


Train 2:   0%|          | 0/3271 [00:00<?, ?it/s]/tmp/ipykernel_19/888885370.py:134: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Train 2: 100%|██████████| 3271/3271 [02:47<00:00, 19.51it/s]


...

Epoch 99 Val Loss 0.1442 MAE 19.5444


Train 100: 100%|██████████| 3271/3271 [02:47<00:00, 19.53it/s]


Epoch 100 Train Loss 0.0008 MAE 1.6806


Val: 100%|██████████| 185/185 [00:08<00:00, 22.39it/s]

Epoch 100 Val Loss 0.1446 MAE 19.5636
Best Val MAE: 18.4396
